<a href="https://colab.research.google.com/github/kenpurin/pyspark/blob/main/test_pyspark.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [29]:
!pip install pyspark


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [30]:
from pyspark import SparkContext
sc = SparkContext.getOrCreate()

In [31]:
from pyspark.sql import SparkSession
# ในกรณีที่ไม่มี spark context อยู่ก่อนแล้ว เราก็สร้างใหม่
if (sc is None):
    sc = SparkContext(master="local[*]")

spark = SparkSession(sparkContext=sc)\
        .builder\
        .appName("How to Spark")\
        .config("spark.some.config.option", "some-value") \
        .getOrCreate()

In [33]:
sc

<SparkContext master=local[*] appName=pyspark-shell>

In [32]:
sc.version

'3.3.0'

In [34]:
#get data test
!git clone https://github.com/codeforamerica/ohana-api.git

fatal: destination path 'ohana-api' already exists and is not an empty directory.


In [35]:
#/content/ohana-api/data/sample-csv/taxonomy.csv
mtcars_rdd = sc.textFile('/content/ohana-api/data/sample-csv/taxonomy.csv')

In [36]:
df = spark.read.csv('/content/ohana-api/data/sample-csv/taxonomy.csv', header=True, inferSchema=True)

In [37]:
df.dtypes

[('taxonomy_id', 'string'),
 ('name', 'string'),
 ('parent_id', 'string'),
 ('parent_name', 'string')]

In [38]:
df.show()

+-----------+--------------------+---------+----------------+
|taxonomy_id|                name|parent_id|     parent_name|
+-----------+--------------------+---------+----------------+
|        101|           Emergency|     null|            null|
|     101-01|   Disaster Response|      101|       Emergency|
|     101-02|      Emergency Cash|      101|       Emergency|
|  101-02-01|   Help Pay for Food|   101-02|  Emergency Cash|
|  101-02-02|Help Pay for Heal...|   101-02|  Emergency Cash|
|  101-02-03|Help Pay for Housing|   101-02|  Emergency Cash|
|  101-02-04|    Help Pay for Gas|   101-02|  Emergency Cash|
|  101-02-05| Help Pay for School|   101-02|  Emergency Cash|
|  101-02-06|Help Pay for Util...|   101-02|  Emergency Cash|
|     101-03|      Emergency Food|      101|       Emergency|
|     101-04|   Emergency Shelter|      101|       Emergency|
|     101-05|Help Find Missing...|      101|       Emergency|
|     101-06|    Immediate Safety|      101|       Emergency|
|  101-0

In [39]:
df.head(10)

[Row(taxonomy_id='101', name='Emergency', parent_id=None, parent_name=None),
 Row(taxonomy_id='101-01', name='Disaster Response', parent_id='101', parent_name='Emergency'),
 Row(taxonomy_id='101-02', name='Emergency Cash', parent_id='101', parent_name='Emergency'),
 Row(taxonomy_id='101-02-01', name='Help Pay for Food', parent_id='101-02', parent_name='Emergency Cash'),
 Row(taxonomy_id='101-02-02', name='Help Pay for Healthcare', parent_id='101-02', parent_name='Emergency Cash'),
 Row(taxonomy_id='101-02-03', name='Help Pay for Housing', parent_id='101-02', parent_name='Emergency Cash'),
 Row(taxonomy_id='101-02-04', name='Help Pay for Gas', parent_id='101-02', parent_name='Emergency Cash'),
 Row(taxonomy_id='101-02-05', name='Help Pay for School', parent_id='101-02', parent_name='Emergency Cash'),
 Row(taxonomy_id='101-02-06', name='Help Pay for Utilities', parent_id='101-02', parent_name='Emergency Cash'),
 Row(taxonomy_id='101-03', name='Emergency Food', parent_id='101', parent_nam

In [40]:
df.first()

Row(taxonomy_id='101', name='Emergency', parent_id=None, parent_name=None)

In [41]:
df.take(2)

[Row(taxonomy_id='101', name='Emergency', parent_id=None, parent_name=None),
 Row(taxonomy_id='101-01', name='Disaster Response', parent_id='101', parent_name='Emergency')]

In [42]:
df.printSchema()

root
 |-- taxonomy_id: string (nullable = true)
 |-- name: string (nullable = true)
 |-- parent_id: string (nullable = true)
 |-- parent_name: string (nullable = true)



In [43]:
df.describe().show() 

+-------+---------------+----------------+------------------+--------------------+
|summary|    taxonomy_id|            name|         parent_id|         parent_name|
+-------+---------------+----------------+------------------+--------------------+
|  count|            290|             290|               279|                 279|
|   mean|          106.0|            null|105.86666666666666|                null|
| stddev|3.3166247903554|            null| 3.306217392478572|                null|
|    min|            101|         12-Step|               101|Addiction & Recovery|
|    max|         111-05|Workplace Rights|         111-01-01|                Work|
+-------+---------------+----------------+------------------+--------------------+



In [44]:
df.describe(['taxonomy_id']).show()

+-------+---------------+
|summary|    taxonomy_id|
+-------+---------------+
|  count|            290|
|   mean|          106.0|
| stddev|3.3166247903554|
|    min|            101|
|    max|         111-05|
+-------+---------------+



In [45]:
df.columns 

['taxonomy_id', 'name', 'parent_id', 'parent_name']

In [46]:
df.count() 

290

In [47]:
spark.stop()